In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Flatten_DF").getOrCreate()
print(spark.sparkContext.appName)

Flatten_DF


In [3]:
from sparkaid import flatten as flat
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
path = "C:/Users/PETERCHAPAGAIN/Downloads/vericred_ambetter/vericred_ambetter.json"
#path = "C:/pyqt/data/imperial_health_plan.json"
df = spark.read.json(path=path)

In [5]:
df.printSchema()
df.count()

root
 |-- addresses: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- address_string: string (nullable = true)
 |    |    |-- city: string (nullable = true)
 |    |    |-- languages: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- name: string (nullable = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |-- office_name: string (nullable = true)
 |    |    |-- phones: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- type: string (nullable = true)
 |    |    |    |    |-- value: string (nullable = true)
 |    |    |-- state: string (nullable = true)
 |    |    |-- street_line_1: string (nullable = true)
 |    |    |-- street_line_2: string (nullable = true)
 |    |    |-- zip: string (nullable = true)
 |-- group_affiliations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- hospital_a

723239

In [6]:
df.show(5)

+--------------------+------------------+---------------------+--------------------+--------------------+--------------------+
|           addresses|group_affiliations|hospital_affiliations|            networks|            provider|         specialties|
+--------------------+------------------+---------------------+--------------------+--------------------+--------------------+
|[{16870 Mary Mart...|                []|                   []|[{Allwell Dual Me...|{true, Aa Mobilit...|[{Durable Medical...|
|[{849 57th Street...|                []|                   []|[{Allwell Dual Me...|{true, Aa Pharmac...|        [{Pharmacy}]|
|[{2610 S Intersta...|                []|                   []|[{Allwell Dual Me...|{true, null, null...|   [{Ophthalmology}]|
|[{313 East 12th S...|                []| [{Dell Seton Medi...|[{Allwell Dual Me...|{true, null, null...|[{Hospitalist}, {...|
|[{1601 Trinity St...|                []|                   []|[{Allwell Dual Me...|{true, null, null...|      

In [7]:
df.select('provider.unparsed_name').show(3)

+--------------+
| unparsed_name|
+--------------+
|   Aa Mobility|
|   Aa Pharmacy|
|Aaron B Roller|
+--------------+
only showing top 3 rows



In [8]:
df.select('addresses.zip').show(3,truncate=False)

+----------------------------+
|zip                         |
+----------------------------+
|[75762]                     |
|[11220]                     |
|[78704, 78759, 78665, 78640]|
+----------------------------+
only showing top 3 rows



In [9]:
df.select('addresses.address_string').show(3,truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|address_string                                                                                                                                                                                    |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[16870 Mary Martin Drive, Flint, TX 75762]                                                                                                                                                        |
|[849 57th Street Floor 1, Brooklyn, NY 11220]                                                                                                                                                     |
|[2610 S Inters

In [87]:
siteuid = df.select('provider.site_uid').distinct()

In [89]:
siteuid.count()

459889

In [90]:
siteuid.show(15)

+--------+
|site_uid|
+--------+
|13817337|
| 8019669|
|15179502|
| 6325543|
| 4134026|
|12890778|
| 5181081|
| 4494067|
| 4495659|
|11786351|
|11793464|
|11812093|
| 5094199|
|11029730|
|11385253|
+--------+
only showing top 15 rows



In [92]:
df.filter(col("provider.site_uid")==4134026).select(['addresses','networks','provider','specialties']).show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------+
|addresses                                                                                                                                                              |networks                                     |provider                                                                                                                                                                                                                                    |specialties                      |
+-------------------------

In [10]:
flatdf = flat(df)

In [11]:
flatdf.printSchema()

root
 |-- addresses_address_string: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- addresses_city: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- addresses_languages_name: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- addresses_languages_type: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- addresses_office_name: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- addresses_phones_type: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- addresses_phones_value: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)
 |-- addresses_state: array (nullable = true)
 |    |-- element: string (containsNul

In [45]:
flatdf.select('addresses_address_string').show(truncate=True)

+------------------------+
|addresses_address_string|
+------------------------+
|    [16870 Mary Marti...|
|    [849 57th Street ...|
|    [2610 S Interstat...|
|    [313 East 12th St...|
|    [1601 Trinity Str...|
|    [3201 West Highwa...|
|    [3801 N Lamar Bou...|
|    [8666 Huebner Roa...|
|    [9902 Potranco Ro...|
|    [803 Castroville ...|
|    [11113 Research B...|
|    [11113 Research B...|
|    [14058 Bee Cave P...|
|    [12129 Ranch Road...|
|    [109 Cypres Creek...|
|    [143 Exchange Bou...|
|    [7330 San Pedro A...|
|    [1448 E Common St...|
|    [3201 S Austin Av...|
|    [1850 W State Hig...|
+------------------------+
only showing top 20 rows



In [56]:
aaron = flatdf.filter(col("provider_unparsed_name")=="Aaron B Roller")\
.select('addresses_address_string','addresses_street_line_1','addresses_zip','hospital_affiliations_name',
        'group_affiliations','specialties_name','networks_name','networks_tier')\
.cache()
#.show(truncate=False)
#.distinct()

In [37]:
aaron.first()

Row(addresses_street_line_1=['2610 S Interstate 35', '3807 Spicewood Springs Road Suite 101', '4010 Sandy Brook Drive', '5401 Fm 1626 Suite 365'], addresses_zip=['78704', '78759', '78665', '78640'], hospital_affiliations_name=[], group_affiliations=[], specialties_name=['Ophthalmology'], networks_name=['Allwell Dual Medicare Harmony HMO D-SNP'], networks_tier=[''])

In [38]:
len(aaron.head(5))>0

True

In [16]:
aaron.count() #try len

15

In [39]:
aaron.printSchema()

root
 |-- addresses_street_line_1: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- addresses_zip: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- hospital_affiliations_name: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- group_affiliations: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- specialties_name: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- networks_name: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- networks_tier: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [83]:
aaron.select(
    col('addresses_street_line_1').getItem(0),
    col('addresses_zip').getItem(0),
    col('specialties_name').getItem(0),
    col('networks_name').getItem(0),col('networks_tier').getItem(0)     
).show(truncate=False)

+-------------------------------------+----------------+-------------------+---------------------------------------+----------------+
|addresses_street_line_1[0]           |addresses_zip[0]|specialties_name[0]|networks_name[0]                       |networks_tier[0]|
+-------------------------------------+----------------+-------------------+---------------------------------------+----------------+
|2610 S Interstate 35                 |78704           |Ophthalmology      |Allwell Dual Medicare Harmony HMO D-SNP|                |
|2610 S Interstate 35                 |78704           |Ophthalmology      |Allwell Dual Medicare HMO SNP          |                |
|3807 Spicewood Springs Road Suite 101|78759           |Ophthalmology      |Allwell Medicare Boost HMO             |                |
|3807 Spicewood Springs Road Suite 101|78759           |Ophthalmology      |Allwell Medicare Complement HMO        |                |
|3807 Spicewood Springs Road Suite 101|78759           |Ophtha

In [26]:
aaron.select('networks_name',size(col('networks_name'))).show()

+--------------------+-------------------+
|       networks_name|size(networks_name)|
+--------------------+-------------------+
|[Allwell Dual Med...|                  1|
|[Allwell Dual Med...|                  1|
|[Allwell Medicare...|                  1|
|[Allwell Medicare...|                  1|
|[Allwell Medicare...|                  1|
|[Allwell Medicare...|                  1|
|[Allwell Medicare...|                  1|
|              [CHIP]|                  1|
|     [CHIP PERINATE]|                  1|
|          [CHIP RSA]|                  1|
|              [STAR]|                  1|
|       [STAR Health]|                  1|
|         [STAR Kids]|                  1|
|         [STAR+PLUS]|                  1|
|     [STAR+PLUS MMP]|                  1|
+--------------------+-------------------+



In [44]:
aaron.select('addresses_street_line_1',size('addresses_street_line_1').alias('total_address'),
             'networks_name',size(col('networks_name').alias('total_network'))).show()

+-----------------------+-------------+--------------------+------------------------------------+
|addresses_street_line_1|total_address|       networks_name|size(networks_name AS total_network)|
+-----------------------+-------------+--------------------+------------------------------------+
|   [2610 S Interstat...|            4|[Allwell Dual Med...|                                   1|
|   [2610 S Interstat...|            4|[Allwell Dual Med...|                                   1|
|   [3807 Spicewood S...|            4|[Allwell Medicare...|                                   1|
|   [3807 Spicewood S...|            4|[Allwell Medicare...|                                   1|
|   [3807 Spicewood S...|            4|[Allwell Medicare...|                                   1|
|   [2610 S Interstat...|            4|[Allwell Medicare...|                                   1|
|   [3807 Spicewood S...|            4|[Allwell Medicare...|                                   1|
|   [2610 S Intersta

In [57]:
aaron.columns

['addresses_address_string',
 'addresses_street_line_1',
 'addresses_zip',
 'hospital_affiliations_name',
 'group_affiliations',
 'specialties_name',
 'networks_name',
 'networks_tier']

In [58]:
aaron.select('addresses_address_string','hospital_affiliations_name','specialties_name','networks_name').distinct().count()

15

In [62]:
aaron.select('addresses_address_string','specialties_name','networks_name').distinct().show(truncate=False)
aaron.select('addresses_address_string','specialties_name','networks_name').distinct().count()

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------------------------------------+
|addresses_address_string                                                                                                                                                                          |specialties_name|networks_name                            |
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------+-----------------------------------------+
|[2610 S Interstate 35, Austin, TX 78704, 3807 Spicewood Springs Road Suite 101, Austin, TX 78759, 4010 Sandy Brook Drive, Suite 105, Round Rock, TX 78665, 5401 Fm 1626 Suite 365, Kyle, TX 78640]|[Ophthalmology] |[Allwell Dual Medic

15

In [48]:
street1 = aaron.select('addresses_street_line_1')

In [55]:
street1.select('addresses_street_line_1').show(truncate=False)

+-------------------------------------------------------------------------------------------------------------+
|addresses_street_line_1                                                                                      |
+-------------------------------------------------------------------------------------------------------------+
|[2610 S Interstate 35, 3807 Spicewood Springs Road Suite 101, 4010 Sandy Brook Drive, 5401 Fm 1626 Suite 365]|
|[2610 S Interstate 35, 3807 Spicewood Springs Road Suite 101, 4010 Sandy Brook Drive, 5401 Fm 1626 Suite 365]|
|[3807 Spicewood Springs Road Suite 101, 2610 S Interstate 35, 4010 Sandy Brook Drive, 5401 Fm 1626 Suite 365]|
|[3807 Spicewood Springs Road Suite 101, 2610 S Interstate 35, 4010 Sandy Brook Drive, 5401 Fm 1626 Suite 365]|
|[3807 Spicewood Springs Road Suite 101, 2610 S Interstate 35, 4010 Sandy Brook Drive, 5401 Fm 1626 Suite 365]|
|[2610 S Interstate 35, 3807 Spicewood Springs Road Suite 101, 4010 Sandy Brook Drive, 5401 Fm 1626 Suit

In [54]:
street1.select('addresses_street_line_1').distinct().show(truncate=False)

+-------------------------------------------------------------------------------------------------------------+
|addresses_street_line_1                                                                                      |
+-------------------------------------------------------------------------------------------------------------+
|[2610 S Interstate 35, 3807 Spicewood Springs Road Suite 101, 4010 Sandy Brook Drive, 5401 Fm 1626 Suite 365]|
|[3807 Spicewood Springs Road Suite 101, 2610 S Interstate 35, 4010 Sandy Brook Drive, 5401 Fm 1626 Suite 365]|
|[2610 S Interstate 35, 5401 Fm 1626 Suite 365, 4010 Sandy Brook Drive, 3807 Spicewood Springs Road Suite 101]|
|[5401 Fm 1626 Suite 365, 2610 S Interstate 35, 4010 Sandy Brook Drive, 3807 Spicewood Springs Road Suite 101]|
|[4010 Sandy Brook Drive, 3807 Spicewood Springs Road Suite 101, 5401 Fm 1626 Suite 365, 2610 S Interstate 35]|
|[2610 S Interstate 35, 3807 Spicewood Springs Road Suite 101, 5401 Fm 1626 Suite 365, 4010 Sandy Brook 

In [72]:
# streets = aaron.select("addresses_street_line_1").withColumn("streets",explode('addresses_street_line_1')).drop("addresses_street_line_1")

In [82]:
streets = df.select(flatten("addresses.languages").alias('language'))

In [83]:
streets.printSchema()

root
 |-- language: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- name: string (nullable = true)
 |    |    |-- type: string (nullable = true)



In [85]:
streets.select('*').show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|language                    

In [41]:
del flataaron

In [42]:
flataaron

NameError: name 'flataaron' is not defined

In [43]:
aaron.unpersist()

DataFrame[addresses_street_line_1: array<string>, addresses_zip: array<string>, hospital_affiliations_name: array<string>, group_affiliations: array<string>, specialties_name: array<string>, networks_name: array<string>, networks_tier: array<string>]